In [7]:
# 필요한 모듈 임포트
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.messages import HumanMessage, AIMessage
from langchain_ollama import OllamaLLM

# LLM을 호출하는 노드 함수 정의
def run_llm(state: MessagesState):
    # 마지막 메시지를 가져와 문자형이면 HumanMessage로 변환
    last_msg = state["messages"][-1]
    if isinstance(last_msg, str):
        user_msg = HumanMessage(content=last_msg)
    else:
        user_msg = last_msg

    # Ollama 모델 호출 (여기서는 'llama3' 모델을 예시로 사용)
    model = OllamaLLM(model="llama3")
    # 모델 호출 결과는 문자열로 반환된다고 가정
    llm_response = model.invoke(user_msg.content)
    # 모델 응답을 AIMessage 객체로 감싸기
    ai_msg = AIMessage(content=llm_response)
    
    # 기존 메시지 리스트에 응답 메시지를 추가하여 반환
    return {"messages": state["messages"] + [ai_msg]}

# 그래프 빌더 생성 (MessagesState를 상태 스키마로 사용)
graph_builder = StateGraph(MessagesState)

# 노드 등록
graph_builder.add_node("llm", run_llm)

# START와 END 상수를 활용하여 그래프의 엣지 설정
graph_builder.add_edge(START, "llm")
graph_builder.add_edge("llm", END)

# 그래프 컴파일
graph = graph_builder.compile()


In [8]:
# 초기 상태 설정 - 메시지는 LangChain 메시지 객체 형식으로 전달할 수도 있지만, 간단히 문자열을 넣어도 됩니다.
initial_state = {"messages": ["What is quantum physics?"]}

# invoke() 메서드로 그래프 실행 (run() 대신 사용)
result = graph.invoke(initial_state)
print(result["messages"])


ConnectError: [Errno 111] Connection refused